https://github.com/v7labs/covid-19-xray-dataset
https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/code?resource=download

In [39]:
!pip install --upgrade sagemaker
!pip3 install -U pip
!pip3 install -U setuptools wheel
!pip3 install autogluon

     |████████████████████████████████| 530 kB 22.3 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.95.0-py2.py3-none-any.whl size=745195 sha256=97ab4b377ec7729b3767a731014fb7c737c48c4f29770cd0eeda49389f6ff735
  Stored in directory: /home/ec2-user/.cache/pip/wheels/11/6c/76/62bf2be73c7e0bf50a86f4732cb2449617fba983fcd5ea3079
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.94.0
    Uninstalling sagemaker-2.94.0:
      Successfully uninstalled sagemaker-2.94.0


In [40]:
import os
import sagemaker
import boto3
from scripts.ag_model import (
    AutoGluonTraining,
    AutoGluonInferenceModel)
from sagemaker import utils

In [41]:
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session._region_name

bucket = sagemaker_session.default_bucket()
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"
output_path = f"s3://{bucket}/{s3_prefix}/output/"

In [42]:
%%writefile scripts/imagepredictortrain.py
import argparse
import os
import autogluon.core as ag
from autogluon.vision import ImageDataset, ImagePredictor
import zipfile
import boto3


def get_env_if_present(name):
    result = None
    if name in os.environ:
        result = os.environ[name]
    return result


def get_input_path(path):
    file = os.listdir(path)[0]
    if len(os.listdir(path)) > 1:
        print(f"WARN: more than one file is found in {channel} directory")
    print(f"Using {file}")
    filename = f"{path}/{file}"
    return filename
    
if __name__ == "__main__":
# Disable Autotune
    os.environ["MXNET_CUDNN_AUTOTUNE_DEFAULT"] = "0"

    # ------------------------------------------------------------ Args parsing
    print("Starting AG")
    parser = argparse.ArgumentParser()

    # Data, model, and output directories
    parser.add_argument(
        "--output-data-dir", type=str, default=get_env_if_present("SM_OUTPUT_DATA_DIR")
    )
    parser.add_argument("--model-dir", type=str, default=get_env_if_present("SM_MODEL_DIR"))
    parser.add_argument("--n_gpus", type=str, default=get_env_if_present("SM_NUM_GPUS"))
    parser.add_argument("--training_dir", type=str, default=get_env_if_present("SM_CHANNEL_TRAIN"))
    parser.add_argument(
        "--test_dir", type=str, required=False, default=get_env_if_present("SM_CHANNEL_TEST")
    )
    parser.add_argument("--ag_config", type=str, default=get_env_if_present("SM_CHANNEL_CONFIG"))

    args, _ = parser.parse_known_args()

    print(f"Args: {args}")
    
     # ---------------------------------------------------------------- Training

    train_file = get_input_path(args.training_dir)
    
    with zipfile.ZipFile(train_file, 'r') as zip_ref:
        zip_ref.extractall('data') 
    train_data,val_data, _= ImageDataset.from_folders('data', train='train', val='val')
    print('train #', len(train_data), 'val #', len(val_data))
    predictor = ImagePredictor()
    predictor.fit(train_data,val_data)
    predictor.save(os.path.join(args.model_dir, "model.ag"))

Overwriting scripts/imagepredictortrain.py


In [43]:
ag = AutoGluonTraining(
    role=role,
    entry_point="scripts/imagepredictortrain.py",
    region=region,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    framework_version="0.3.1",
    py_version='py37'
)

In [44]:
ag.fit({ "train": "s3://aws-experiments-b18961/6/chest_xray_train.zip"})

2022-06-20 18:10:35 Starting - Starting the training job...
2022-06-20 18:11:02 Starting - Preparing the instances for trainingProfilerReport-1655748634: InProgress
.........
2022-06-20 18:12:31 Downloading - Downloading input data......
2022-06-20 18:13:31 Training - Downloading the training image.....................
2022-06-20 18:16:53 Training - Training image download completed. Training in progress.This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.download.nvidia.com/licenses/NVIDIA_Deep_Learning_Container_License.pdf
2022-06-20 18:16:56,075 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2022-06-20 18:16:56,097 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': 'i

In [45]:
sm_boto3 = boto3.client('sagemaker')
artifact = sm_boto3.describe_training_job(
    TrainingJobName=ag.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)
!aws s3 cp $artifact model/
! tar -xf model/model.tar.gz -C model/

Model artifact persisted at s3://sagemaker-us-east-1-485822383573/autogluon-training-2022-06-20-18-10-34-550/output/model.tar.gz


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


download: s3://sagemaker-us-east-1-485822383573/autogluon-training-2022-06-20-18-10-34-550/output/model.tar.gz to model/model.tar.gz


In [46]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset

In [47]:
predictor= ImagePredictor.load('model/model.ag')
_, _, test_dataset = ImageDataset.from_folders('data', test='test')
print(test_dataset)

                                                 image  label
0    /home/ec2-user/SageMaker/experiments/6/data/te...      0
1    /home/ec2-user/SageMaker/experiments/6/data/te...      0
2    /home/ec2-user/SageMaker/experiments/6/data/te...      0
3    /home/ec2-user/SageMaker/experiments/6/data/te...      0
4    /home/ec2-user/SageMaker/experiments/6/data/te...      0
..                                                 ...    ...
619  /home/ec2-user/SageMaker/experiments/6/data/te...      1
620  /home/ec2-user/SageMaker/experiments/6/data/te...      1
621  /home/ec2-user/SageMaker/experiments/6/data/te...      1
622  /home/ec2-user/SageMaker/experiments/6/data/te...      1
623  /home/ec2-user/SageMaker/experiments/6/data/te...      1

[624 rows x 2 columns]


In [48]:
predictor.predict(test_dataset)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0      0
1      0
2      0
3      0
4      0
      ..
619    1
620    1
621    1
622    1
623    0
Name: label, Length: 624, dtype: int64

In [49]:
test_acc = predictor.evaluate(test_dataset)
print('Top-1 test acc: %.3f' % test_acc['top1'])

[Epoch 3] validation: top1=0.911859 top5=1.000000


Top-1 test acc: 0.912
